# Transformations de Fourier avec Scipy

La transformation de Fourier est un outil mathématique utilisé pour analyser les signaux. Elle décompose un signal et renvoie son spectre, qui révèle les différentes fréquences qui composent le signal et leurs puissances respectives.

Nous allons utiliser le module `scipy.fft`. Il existe d'autres implémentations de la transformation de Fourier.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.fft import fft, fftfreq
from scipy.io import wavfile
from IPython.display import Audio

Nous allons nous amuser à créer des signaux sonores et à les faire jouer par le notebook.

Typiquement, un signal sonore est enregistré par echantillonnage à une certaine fréquence, ce qui signifie que du signal analogique initial, on prélève un certain nombre de valeurs numériques chaque seconde.

Un CD Audio par exemple utilise un échantillonnage à 44 100 Hz, autrement dit 44 100 valeurs numériques par seconde sont utilisées pour enregistrer puis relire le signal.

Nous allons écrire une fonction `note(f)` qui va créer un signal sinusoidal (un array numpy) à partir d'une fréquence donnée, avec comme constantes un taux d'echantillonnage en Hz `R` et une durée du signal en secondes `D`

In [ ]:
D = 3  # durée du signal produit
R = 44100  # échantillonnage en Hz


def note(f):
    # t représente le temps, de 0 à la durée D, avec R*D points
    t = np.linspace(0, D, R * D)
    signal = np.sin(2 * np.pi * f * t)
    return signal

Regardons les 1000 premiers points d’une note produite avec une fréquence de 440 Hz.

Comme nous allons reproduire le même type de graphe plusieurs fois, c'est une bonne idée de définir une fonction qui nous évitera de répéter les mêmes réglages à plusieurs reprises.

In [ ]:
def plot_signal(signal, samples=1000):
    plt.plot(signal[:samples])
    plt.xlabel("temps")
    plt.ylabel("amplitude")

In [ ]:
plot_signal(note(440))

Maintenant nous allons stocker ce signal dans une variable et le jouer !

In [ ]:
la = note(440)
Audio(la, rate=R)

On peut aussi créer plusieurs notes et les combiner en un accord.
Par exemple :
- une note de base comme notre `la` à 440 Hz, 
- combinée à sa tierce majeure, dont la fréquence est  $\frac{5}{4}$ fois plus grande
- et sa quinte, dont la fréquence est $\frac{3}{2}$ fois plus grande

On donnera à chaque note un coefficient pour que notre note de base domine l'accord, que la quinte ait son amplitude diminuée de moitiée, et ne garder qu'un quart de l'amplitude de la tierce.

In [ ]:
base_freq = 440

base_note = note(base_freq)
fifth_note = note(base_freq * 3 / 2)
major_third = note(base_freq * 5 / 4)

chord = base_note + 0.5 * fifth_note + 0.25 * major_third

Audio(chord, rate=R)

Regardons les 1000 premiers échantillons de notre accord

In [ ]:
plot_signal(chord)

## [Transformation de Fourier rapide](https://fr.wikipedia.org/wiki/Transformation_de_Fourier_rapide)

> Cet algorithme est couramment utilisé en traitement numérique du signal pour transformer des données discrètes du domaine temporel dans le domaine fréquentiel […]

![](fft.png)

Utilisons le module `scipy.fft` pour analyser les signaux que nous avons créés plus haut.

Exécutez la cellule ci-dessous puis remontez, modifiez votre accord, et essayez à nouveau la cellule ci-dessous ?

Que constatez-vous ?

In [ ]:
def plot_fft(signal, rate, freq_max):
    Y = np.abs(fft(signal))
    X = np.abs(fftfreq(Y.size) * rate)
    plt.xlim(0, freq_max)
    plt.xlabel("Fréquence")
    plt.ylabel("Amplitude")
    plt.plot(X, Y)


plot_fft(chord, R, 2000)

## Exemples de signaux réels

Essayons de reproduire cette décomposition sur des signaux moins artifiels.

Un certain nombre de fichiers `.wav` sont fournis dans ce dossier.

In [ ]:
!ls -lh *.wav

Commeçons par regarder à quoi ressemble le contenu du fichier `bird.wav`.

In [ ]:
rate, audio = wavfile.read("bird.wav")

In [ ]:
audio

`audio` contient un array à 2 dimensions. Chaque ligne représente un échantillonnage dans le temps. 

Le signal est stéréo, les deux colonnes représentent les côtés gauche et droit.

Pour donner au signal la forme qui nous intéresse, nous allons simplement transposer cet array (chaque colonne devient une ligne) puis sélectionner arbitrairement la première ligne, soit le signal côté gauche, avec `audio.T[0]`.

Regardons le contenu d'une tranche de 10 échantillons.

In [ ]:
audio.T[0][R : (R + 10)]

In [ ]:
plot_signal(audio.T[0], len(audio))

Écoutez ce signal. Est-ce que ce l'amplitude dans le temps correspond bien ?

In [ ]:
Audio(data=audio.T[0], rate=rate)

Quelles sont les fréquences dominantes ?

In [ ]:
plot_fft(audio.T[0], rate, 7500)

---
À vous maintenant !
Reproduisez cette analyse et cette écoute avec les autres fichiers `.wav` fournis.